Here, I'm going to find out new parameter for win.
When I played football, our coaches often said us "Play decisively!". However, nobody understood what "decisively" was.
So I want to quantify "decisively".
This competition provide us a variety of data, and I remark head direction and running direction.
I have hypothesis that head direction and running direction are related to gain yard and "decisively" means rate of concordance of head direction and running direction.

import data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
game = pd.read_csv("../input/nfl-big-data-bowl-2021/games.csv")
players = pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")
plays = pd.read_csv("../input/nfl-big-data-bowl-2021/plays.csv")
week1 = pd.read_csv("../input/nfl-big-data-bowl-2021/week1.csv")
week2 = pd.read_csv("../input/nfl-big-data-bowl-2021/week2.csv")
week3 = pd.read_csv("../input/nfl-big-data-bowl-2021/week3.csv")
week4 = pd.read_csv("../input/nfl-big-data-bowl-2021/week4.csv")
week5 = pd.read_csv("../input/nfl-big-data-bowl-2021/week5.csv")
week6 = pd.read_csv("../input/nfl-big-data-bowl-2021/week6.csv")
week7 = pd.read_csv("../input/nfl-big-data-bowl-2021/week7.csv")
week8 = pd.read_csv("../input/nfl-big-data-bowl-2021/week8.csv")
week9 = pd.read_csv("../input/nfl-big-data-bowl-2021/week9.csv")
week10 = pd.read_csv("../input/nfl-big-data-bowl-2021/week10.csv")
week11 = pd.read_csv("../input/nfl-big-data-bowl-2021/week11.csv")
week12 = pd.read_csv("../input/nfl-big-data-bowl-2021/week12.csv")
week13 = pd.read_csv("../input/nfl-big-data-bowl-2021/week13.csv")
week14 = pd.read_csv("../input/nfl-big-data-bowl-2021/week14.csv")
week15 = pd.read_csv("../input/nfl-big-data-bowl-2021/week15.csv")
week16 = pd.read_csv("../input/nfl-big-data-bowl-2021/week16.csv")
week17 = pd.read_csv("../input/nfl-big-data-bowl-2021/week17.csv")

In [ ]:
players['height'] = players['height'].str.replace('-', '.')
players['height'] = players['height'].astype(np.float32)
players.loc[players['height']>10, 'height'] /= 12

check summary information

In [ ]:
game.info()

In [ ]:
players.info()

In [ ]:
plays.info()

In [ ]:
week1.info()

data analyse

analysis of speed data

I forcus on week1 data.

In [ ]:
week1_speed = week1[["nflId", "displayName", "s", "a", "o", "dir"]]

In [ ]:
def weekly_speed(weekly_data):
#   only defense player
    weekly_data = weekly_data.query('position == ["SS", "FS", "MLB", "CB", "LB", "FB", "OLB", "ILB", "DL", "DB", "NT", "S", "DE"]')
    return weekly_data[["nflId", "displayName", "s", "a", "o", "dir", "playId"]]

In [ ]:
playResult = plays[["gameId","playId", "playResult", "possessionTeam"]]

In [ ]:
week1_speed = weekly_speed(week1)

In [ ]:
week1_speed = pd.merge(week1_speed, playResult, how='left', on="playId")

In [ ]:
week1_speed = pd.merge(week1_speed, game, on="gameId")

extract defense team

In [ ]:
week1_speed.loc[week1_speed["possessionTeam"] == week1_speed["homeTeamAbbr"], "defTeam"] = week1_speed["visitorTeamAbbr"]
week1_speed.loc[week1_speed["possessionTeam"] != week1_speed["homeTeamAbbr"], "defTeam"] = week1_speed["homeTeamAbbr"]

delete ball data

In [ ]:
week1_speed = week1_speed[week1_speed["displayName"] != "Football"]

In [ ]:
week1_speed["playResult"].hist(bins=30)

This data is almost pass data, and failed pass means 0 yard gain.

fastest player in week1

In [ ]:
week1_speed.loc[week1_speed.groupby("displayName")["s"].idxmax(),:].sort_values("s", ascending=False).head(10)[["displayName", "s"]]

calculate facing angle against running angle(relative_angle)

In [ ]:
def calculate_relative_speed(weekly_data):
    weekly_data["relative_angle"] = abs(weekly_data["dir"] - weekly_data["o"])
    weekly_data.loc[abs(weekly_data["dir"] - weekly_data["o"]) > 180, "relative_angle"] = 360 - abs(weekly_data["dir"] - weekly_data["o"])
    weekly_data["relative_angle"] = weekly_data["relative_angle"].round()
    return weekly_data

In [ ]:
week1_speed = calculate_relative_speed(week1_speed)

max speed of every relative angle

In [ ]:
week1_speed["relative_angle"].hist(bins=180)

This means players run front face many time.

In [ ]:
r_speed_max = week1_speed.loc[week1_speed.groupby("relative_angle")["s"].idxmax(),:]

avarage speed by relative angle

In [ ]:
week1_speed.groupby("relative_angle").agg({"s": 'mean'}).plot(title="average speed by angle")

This means relative angle and avarage speed have negative corelation.

average acceleration by relative angle

In [ ]:
week1_speed.groupby("relative_angle").agg({"a": 'mean'}).plot(title="average speed by angle")

Check playResult and relative_angle

In [ ]:
week1_res_dir = week1_speed.groupby("playId").agg({"playResult": 'mean', "relative_angle": 'mean'})

In [ ]:
week1_res_dir = week1_res_dir.reset_index()

In [ ]:
week1_res_dir["relative_angle"].hist(bins=100)

Relative angle has gauss distribution.

In [ ]:
week1_res_dir.plot.scatter("playResult", y="relative_angle")

This means that I can't find out no relation in relative angle and playResult.

In this competition, I failed to discover new parameter for win. But I still think there is some relation in relative angle and "play decisively".
So, I continue to research research that relation.
Thank you for reading and sorry for poor writing.